In [233]:
puzzle = open("AoC17.txt").read().splitlines()


In [234]:
A = int(puzzle[0].split(": ")[-1])
B = int(puzzle[1].split(": ")[-1])
C = int(puzzle[2].split(": ")[-1])
program = puzzle[4].split(": ")[-1].split(",")
program = [int(i) for i in program]

In [235]:
#instruction (represented by opcode), operand
#so instruction pointer increases by 2 each time (unless it jumps)
#two types of operands - combo and literal
#combo 0-3 is literal values
#combo 4 is register A
#combo 5 is register B
#combo 6 is register C
#combo 7 is reserved (will not appear in valid program)
#instructions:
#0 - adv: calculates value in A/2^(combo operand), truncates to integer, stores in A
#1 - bxl: calculates bitwise XOR of B and literal operand, stores in B
#2 - bst: calculates combo operand mod 8, stores in B
#3 - jnz: if A is not zero, jumps to literal operand (so instruction pointer doesn't increase by 2 after)
#4 - bxc: calculates bitwise XOR of B and C, stores in B (reads operand but doesn't use it)
#5 - out: calculates valye of combo operand mod 8, outputs to terminal
#6 - bdv: calculates value in A/2^(combo operand), truncates to integer, stores in B
#7 - cdv: calculates value in A/2^(combo operand), truncates to integer, stores in C

In [236]:
def comboOp(combo, A, B, C):
    if combo < 4:
        return combo
    elif combo == 4:
        return A
    elif combo == 5:
        return B
    elif combo == 6:
        return C
    return -1
      

In [237]:
def run_program(program, A, B, C):
    ip = 0
    outputs = []
    while ip < len(program):
        instruction = program[ip]
        lit_operand = program[ip+1]
        combo_operand = comboOp(lit_operand, A, B, C)
        if instruction == 0:
            A = A // (2**combo_operand)
        elif instruction == 1:
            B = B ^ lit_operand
        elif instruction == 2:
            B = combo_operand % 8
        elif instruction == 3:
            if A != 0:
                ip = lit_operand
                continue
        elif instruction == 4:
            B = B ^ C
        elif instruction == 5:
            outputs.append(combo_operand % 8)
        elif instruction == 6:
            B = A // (2**combo_operand)
        elif instruction == 7:
            C = A // (2**combo_operand)
        ip += 2
    return A, B, C, outputs

In [238]:
def convertAns(outputs):
    return "".join([str(i)+"," for i in outputs])[:-1]

In [239]:
A_new, B_new, C_new, outputs = run_program(program, A, B, C)
ans1 = convertAns(outputs)
print(ans1)

4,3,7,1,5,3,0,5,4


In [240]:
def checkIfCopy(program, A, B, C):
    initial_A = A
    ip = 0
    outputs = []
    ctr = 0
    while ip < len(program):
        instruction = program[ip]
        lit_operand = program[ip+1]
        combo_operand = comboOp(lit_operand, A, B, C)
        if instruction == 0:
            A = A // (2**combo_operand)
        elif instruction == 1:
            B = B ^ lit_operand
        elif instruction == 2:
            B = combo_operand % 8
        elif instruction == 3:
            if A != 0:
                ip = lit_operand
                continue
        elif instruction == 4:
            B = B ^ C
        elif instruction == 5:
            outputs.append(combo_operand % 8)
            if outputs[-1] != program[ctr]:
                return False
            ctr += 1
        elif instruction == 6:
            B = A // (2**combo_operand)
        elif instruction == 7:
            C = A // (2**combo_operand)
        ip += 2
    return outputs == program

In [241]:
from tqdm import tqdm
def checkPostFix(program, A, B, C, init):
    poss = []
    for i in range(8):
        p = run_program(program, init*8+i, B, C)
        n = len(p[-1])
        last = program[-n:]
        if p[-1] == last:
            poss.append(init*8+i)        
    return poss

In [242]:
possibilities = checkPostFix(program, A, B, C, 5)
for i in tqdm(possibilities):
    print(i, run_program(program, i, B, C))

100%|██████████| 2/2 [00:00<00:00, 20815.40it/s]

43 (0, 0, 0, [3, 0])
47 (0, 0, 0, [3, 0])


In [243]:
programs = []
#use bfs to find all 16 digit programs
queue = [(0,0)] #elements in queue are (init, number of digits in result)
while queue:
    init, n = queue.pop(0)
    if n == 16:
        programs.append(init)
    else:
        poss = checkPostFix(program, A, B, C, init)
        for p in poss:
            queue.append((p, n+1))
ans2 = programs[0]
print(ans2)

190384615275535
